In [1]:
import dspy
import os
from data_models.chapi_unoplat_function import Function
from data_models.chapi_unoplat_function_summary import FunctionSummary
from loader import JsonLoader
from loader import JsonParser

ollama_mistral = dspy.OllamaLocal(model='mistral:7b-instruct-fp16')
dspy.configure(lm=ollama_mistral)

ollama_mistral("We are going to build class summary based on step by step approach. First we will build function by function summary then based on that we will build class level summary. For this user will be sharing relevant metadata in json for each function first and then class.")



class UnoplatFunctionSummary(dspy.Signature):
    """Generate Summary in json based on FunctionSummary"""
    
    input: Function = dspy.InputField(desc="will contain all relevant function metadata")
    output: FunctionSummary = dspy.OutputField(desc="summarisation of function")

cot_predictor = dspy.TypedChainOfThought(UnoplatFunctionSummary)

# read the json file springstaterjava1_codes into pydantic


iload_json = JsonLoader()

iparse_json = JsonParser()


meadata_classes = iload_json.load_json_from_file("springstarterjava1_codes.json")

package_dict = iparse_json.parse_json_to_nodes(meadata_classes,None)

print(package_dict.keys)


function: Function = package_dict['com.datastax.examples.order'][2].functions[0]
print(function)

prediction = cot_predictor(input=function,options={"format": "json"})

ollama_mistral.inspect_history(n=1)

/Users/jayghiya/Documents/unoplat/unoplat-codebase-understanding/unoplat-code-confluence/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-11 14:29:40.940 | INFO     | loader.json_loader:load_json_from_file:28 - JSON data successfully loaded from springstarterjava1_codes.json


<built-in method keys of dict object at 0x165201300>
name='root' return_type='ModelAndView' function_calls=[] annotations=[Annotation(name='RequestMapping', position=Position(start_line=17, start_line_position=4, stop_line=17, stop_line_position=59))] position=Position(start_line=18, start_line_position=11, stop_line=20, stop_line_position=4) local_variables=[FieldModel(type_type='ModelAndView', type_value='return', type_key=None)] body_hash=763749358 content='ModelAndView root() {        return new ModelAndView("redirect:/swagger-ui/");    '
